In [7]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, BatchNormalization, Concatenate, UpSampling2D
from PIL import Image
import gradio as gr
import os
from typing import Tuple, Optional

class FaceAgingApp:
    def __init__(self, model_path: str = None):
        """Initialize the face aging application"""
        # Initialize face detection
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        
        # Age groups configuration
        self.age_groups = ['20-30', '30-40', '40-50', '50-60', '60-70', '70+']
        
        # Load or initialize model
        self.model = self.load_or_create_model(model_path)

    def load_or_create_model(self, model_path: Optional[str]) -> Model:
        """Load pre-trained model or create a new one if not available"""
        if model_path and os.path.exists(model_path):
            print(f"Loading model from {model_path}")
            return load_model(model_path)
        else:
            print("No pre-trained model found. Creating a new model architecture.")
            return self.build_generator()
    
    def build_generator(self) -> Model:
        """Build a generator model for face aging (simplified U-Net)"""
        # Input layers
        input_img = Input(shape=(256, 256, 3))
        age_cond = Input(shape=(len(self.age_groups),))
        
        # Expand age condition to spatial dimensions
        age_cond_expanded = tf.keras.layers.Lambda(
            lambda x: tf.tile(x[:, tf.newaxis, tf.newaxis, :], [1, 256, 256, 1])
        )(age_cond)
        
        # Concatenate image with age condition
        x = Concatenate()([input_img, age_cond_expanded])
        
        # Encoder
        def encoder_block(x, filters, kernel_size=4, strides=2, padding='same'):
            x = Conv2D(filters, kernel_size, strides=strides, padding=padding)(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization()(x)
            return x
        
        # Decoder
        def decoder_block(x, skip_input, filters, kernel_size=4, strides=2, padding='same'):
            x = UpSampling2D(size=2)(x)
            x = Conv2D(filters, kernel_size, strides=1, padding=padding)(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization()(x)
            x = Concatenate()([x, skip_input])
            return x
        
        # Encoder path
        e1 = encoder_block(x, 64)
        e2 = encoder_block(e1, 128)
        e3 = encoder_block(e2, 256)
        e4 = encoder_block(e3, 512)
        
        # Bottleneck
        b = Conv2D(512, 4, strides=2, padding='same')(e4)
        b = LeakyReLU(alpha=0.2)(b)
        
        # Decoder path
        d1 = decoder_block(b, e4, 512)
        d2 = decoder_block(d1, e3, 256)
        d3 = decoder_block(d2, e2, 128)
        d4 = decoder_block(d3, e1, 64)
        
        # Output layer
        output = Conv2D(3, 4, strides=1, padding='same', activation='tanh')(d4)
        
        return Model(inputs=[input_img, age_cond], outputs=output)

    def detect_and_align_face(self, image_path: str) -> Optional[np.ndarray]:
        """Detect and align face from image"""
        try:
            img = cv2.imread(image_path)
            if img is None:
                raise ValueError("Could not read image")
                
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
            
            if len(faces) == 0:
                return None
                
            (x, y, w, h) = max(faces, key=lambda f: f[2]*f[3])
            face = img[y:y+h, x:x+w]
            face = cv2.resize(face, (256, 256))
            face = (face.astype(np.float32) - 127.5) / 127.5
            
            return face
        except Exception as e:
            print(f"Error in face detection: {e}")
            return None

    def predict_age_progression(self, face_image: np.ndarray, target_age_group: str) -> np.ndarray:
        """Predict age progression for a face image"""
        try:
            if face_image is None:
                raise ValueError("No face image provided")
                
            if target_age_group not in self.age_groups:
                raise ValueError(f"Invalid age group. Must be one of {self.age_groups}")
                
            input_img = np.expand_dims(face_image, axis=0)
            age_idx = self.age_groups.index(target_age_group)
            age_cond = np.zeros((1, len(self.age_groups)))
            age_cond[0, age_idx] = 1
            
            aged_face = self.model.predict([input_img, age_cond])
            aged_face = ((aged_face[0] + 1) * 127.5).astype(np.uint8)
            
            return aged_face
        except Exception as e:
            print(f"Error in age prediction: {e}")
            return None

    def process_image(self, input_image: Image.Image, target_age: str) -> Image.Image:
        """Process an uploaded image through the age progression pipeline"""
        try:
            # Save temporary image
            temp_path = "temp_input.jpg"
            input_image.save(temp_path)
            
            # Detect and align face
            face = self.detect_and_align_face(temp_path)
            if face is None:
                return "No face detected. Please upload a clear front-facing photo."
            
            # Generate aged face
            aged_face = self.predict_age_progression(face, target_age)
            if aged_face is None:
                return "Error generating age progression."
            
            # Convert to PIL Image
            aged_pil = Image.fromarray(aged_face)
            
            # Create comparison
            original_resized = Image.open(temp_path).resize((256, 256))
            comparison = Image.new('RGB', (512, 256))
            comparison.paste(original_resized, (0, 0))
            comparison.paste(aged_pil, (256, 0))
            
            # Clean up
            if os.path.exists(temp_path):
                os.remove(temp_path)
                
            return comparison
        except Exception as e:
            print(f"Error in processing: {e}")
            return f"An error occurred: {str(e)}"

def launch_app():
    """Launch the Gradio interface"""
    app = FaceAgingApp("face_aging_model.h5")
    
    iface = gr.Interface(
        fn=app.process_image,
        inputs=[
            gr.Image(label="Upload Your Photo", type="pil"),
            gr.Dropdown(
                choices=app.age_groups,
                label="Target Age Group",
                value="50-60"
            )
        ],
        outputs=gr.Image(label="Age Progression"),
        title="AI Face Aging App",
        description="Upload your photo to see how you might look at different ages.",
        examples=[
            ["example_young.jpg", "40-50"],
            ["example_middle.jpg", "70+"]
        ]
    )
    
    iface.launch()

if __name__ == "__main__":
    launch_app()

No pre-trained model found. Creating a new model architecture.


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [8]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Concatenate
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from PIL import Image
import gradio as gr
import dlib  # More accurate face detection
import face_alignment  # Better alignment
from skimage.transform import estimate_transform, warp
from typing import Tuple, Optional, List
import tempfile
import os

class AdvancedFaceAger:
    def __init__(self, model_path: str = None):
        """Initialize the advanced face aging system"""
        # Initialize face detection and alignment
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
        self.fa = face_alignment.FaceAlignment(
            face_alignment.LandmarksType.TWO_D, 
            flip_input=False,
            device='cuda' if tf.test.is_gpu_available() else 'cpu'
        )
        
        # Age configuration
        self.age_bins = [
            (20, 30), (30, 40), (40, 50), 
            (50, 60), (60, 70), (70, 80)
        ]
        self.age_labels = [f"{start}-{end}" for start, end in self.age_bins]
        
        # Load or create model
        self.generator = self.load_or_build_model(model_path)
        
        # Image processing parameters
        self.img_size = 256
        self.crop_size = 224
        self.scale = 1.6

    def load_or_build_model(self, model_path: Optional[str]) -> Model:
        """Load pre-trained model or build a new one with advanced architecture"""
        if model_path and os.path.exists(model_path):
            print(f"Loading pre-trained model from {model_path}")
            return tf.keras.models.load_model(model_path)
        
        print("Building new model with advanced architecture...")
        return self.build_caae_model()

    def build_caae_model(self) -> Model:
        """Build a Conditional Adversarial Autoencoder (CAAE) model"""
        # Image input
        img_input = Input(shape=(self.crop_size, self.crop_size, 3))
        
        # Age condition input (one-hot encoded)
        age_input = Input(shape=(len(self.age_bins),))
        
        # ----------------------------
        # Encoder (ResNet-like)
        # ----------------------------
        def conv_block(x, filters, kernel_size=3, strides=2, use_leaky=True):
            x = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
            x = BatchNormalization()(x)
            return LeakyReLU(alpha=0.2)(x) if use_leaky else Activation('relu')(x)
        
        # Encoder path
        e1 = conv_block(img_input, 64)  # 112x112
        e2 = conv_block(e1, 128)       # 56x56
        e3 = conv_block(e2, 256)       # 28x28
        e4 = conv_block(e3, 512)       # 14x14
        e5 = conv_block(e4, 512)       # 7x7
        
        # Flatten and concatenate with age
        flattened = Flatten()(e5)
        concat = Concatenate()([flattened, age_input])
        
        # Latent space
        latent = Dense(256, activation='relu')(concat)
        
        # ----------------------------
        # Decoder
        # ----------------------------
        def deconv_block(x, filters, kernel_size=3, strides=2):
            x = Conv2DTranspose(filters, kernel_size, strides=strides, padding='same')(x)
            x = BatchNormalization()(x)
            return LeakyReLU(alpha=0.2)(x)
        
        # Reshape and project
        projected = Dense(7*7*512)(latent)
        reshaped = Reshape((7, 7, 512))(projected)
        
        # Decoder path
        d1 = deconv_block(reshaped, 512)  # 14x14
        d2 = deconv_block(d1, 256)        # 28x28
        d3 = deconv_block(d2, 128)        # 56x56
        d4 = deconv_block(d3, 64)         # 112x112
        d5 = Conv2DTranspose(3, 3, strides=2, padding='same', activation='tanh')(d4)  # 224x224
        
        return Model(inputs=[img_input, age_input], outputs=d5)

    def align_face(self, image: np.ndarray) -> Optional[np.ndarray]:
        """Advanced face alignment using facial landmarks"""
        try:
            # Convert to RGB if needed
            if len(image.shape) == 2:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            elif image.shape[2] == 1:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            elif image.shape[2] == 4:
                image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
            
            # Get facial landmarks
            landmarks = self.fa.get_landmarks(image)
            if not landmarks:
                return None
                
            landmarks = landmarks[0]  # Take first face found
            
            # Calculate transformation
            src_points = np.array([
                landmarks[17], landmarks[21], landmarks[22], 
                landmarks[26], landmarks[36], landmarks[39], 
                landmarks[42], landmarks[45], landmarks[31], 
                landmarks[35], landmarks[48], landmarks[54], 
                landmarks[57], landmarks[8]
            ])
            
            # Estimate similarity transform
            tform = estimate_transform(
                'similarity',
                src_points,
                self.get_reference_landmarks()
            )
            
            # Warp and crop face
            aligned_face = warp(
                image, 
                tform.inverse, 
                output_shape=(self.img_size, self.img_size),
                mode='constant'
            )
            
            # Crop to final size
            margin = (self.img_size - self.crop_size) // 2
            aligned_face = aligned_face[
                margin:margin+self.crop_size,
                margin:margin+self.crop_size
            ]
            
            # Normalize to [-1, 1]
            aligned_face = (aligned_face * 255).astype(np.uint8)
            aligned_face = (aligned_face.astype(np.float32) - 127.5) / 127.5
            
            return aligned_face
        except Exception as e:
            print(f"Error in face alignment: {e}")
            return None

    def get_reference_landmarks(self) -> np.ndarray:
        """Get reference landmarks for alignment"""
        # Standard reference points based on average face
        ref_landmarks = np.array([
            [38.2946, 51.6963],  # Left eye left corner
            [73.5318, 51.5014],  # Right eye right corner
            [56.0252, 71.7366],  # Nose tip
            [41.5493, 92.3655],  # Mouth left corner
            [70.7299, 92.2041]   # Mouth right corner
        ])
        
        # Scale and center based on our image size
        ref_landmarks = ref_landmarks * self.img_size / 112.0
        ref_landmarks = ref_landmarks + (self.img_size - self.crop_size) / 2.0
        
        return ref_landmarks

    def predict_aging(self, face_image: np.ndarray, target_age_idx: int) -> np.ndarray:
        """Predict aging effects with more sophisticated processing"""
        try:
            # Create age condition vector
            age_cond = np.zeros(len(self.age_bins))
            age_cond[target_age_idx] = 1
            age_cond = np.expand_dims(age_cond, axis=0)
            
            # Prepare input
            input_img = np.expand_dims(face_image, axis=0)
            
            # Predict
            aged_face = self.generator.predict([input_img, age_cond])[0]
            
            # Post-processing
            aged_face = (aged_face * 127.5 + 127.5).astype(np.uint8)
            
            # Apply aging artifacts (simulated)
            aged_face = self.add_aging_artifacts(aged_face, target_age_idx)
            
            return aged_face
        except Exception as e:
            print(f"Error in age prediction: {e}")
            return None

    def add_aging_artifacts(self, image: np.ndarray, age_idx: int) -> np.ndarray:
        """Add realistic aging artifacts (wrinkles, spots, etc.)"""
        # Convert to float for processing
        img_float = image.astype(np.float32) / 255.0
        
        # Get age parameters
        age_start, age_end = self.age_bins[age_idx]
        mean_age = (age_start + age_end) / 2
        
        # 1. Wrinkles (intensity increases with age)
        if mean_age > 40:
            # Generate Perlin noise for wrinkles
            noise = self.generate_perlin_noise(image.shape[:2], scale=0.05)
            wrinkle_strength = min(0.15, (mean_age - 40) * 0.005)
            img_float = self.apply_wrinkles(img_float, noise, wrinkle_strength)
        
        # 2. Age spots (appear after 50)
        if mean_age > 50:
            spot_strength = min(0.1, (mean_age - 50) * 0.002)
            img_float = self.apply_age_spots(img_float, spot_strength)
        
        # 3. Skin tone changes (yellowing)
        if mean_age > 60:
            yellowing = min(0.08, (mean_age - 60) * 0.002)
            img_float[:, :, 0] *= (1 - yellowing * 0.5)  # Reduce blue
            img_float[:, :, 1] *= (1 + yellowing * 0.3)  # Increase green slightly
        
        # Convert back to uint8
        return (np.clip(img_float, 0, 1) * 255).astype(np.uint8)

    def generate_perlin_noise(self, shape: Tuple[int, int], scale: float = 0.1) -> np.ndarray:
        """Generate Perlin noise for realistic wrinkles"""
        # Simplified Perlin-like noise
        x = np.linspace(0, scale * shape[1], shape[1], endpoint=False)
        y = np.linspace(0, scale * shape[0], shape[0], endpoint=False)
        xv, yv = np.meshgrid(x, y)
        
        noise = np.sin(xv) * np.cos(yv) * np.sin(xv + yv)
        return (noise - noise.min()) / (noise.max() - noise.min())

    def apply_wrinkles(self, image: np.ndarray, noise: np.ndarray, strength: float) -> np.ndarray:
        """Apply wrinkle effects to image"""
        # Create wrinkle mask (emphasize on forehead and around eyes)
        mask = np.zeros_like(image[:, :, 0])
        h, w = mask.shape
        
        # Forehead area
        mask[:h//3, :] = 1.0
        
        # Eye area
        mask[h//3:h//2, w//4:3*w//4] = 1.0
        
        # Combine with noise
        wrinkle_mask = noise * mask
        
        # Apply to image (darken wrinkles)
        result = image.copy()
        for c in range(3):
            result[:, :, c] = np.clip(
                result[:, :, c] - wrinkle_mask * strength * (0.8 + 0.2 * np.random.random()),
                0, 1
            )
        
        return result

    def process_image_pipeline(self, input_image: Image.Image, target_age: str) -> Image.Image:
        """Complete processing pipeline with better error handling"""
        try:
            with tempfile.NamedTemporaryFile(suffix='.jpg', delete=False) as tmp:
                # Save temporary image
                input_image.save(tmp.name)
                
                # Convert to numpy array
                img_array = np.array(input_image)
                if img_array.shape[-1] == 4:  # RGBA to RGB
                    img_array = img_array[..., :3]
                
                # Align face
                aligned_face = self.align_face(img_array)
                if aligned_face is None:
                    return "Could not detect or align a face. Please try another image."
                
                # Get target age index
                try:
                    age_idx = self.age_labels.index(target_age)
                except ValueError:
                    return f"Invalid age selection. Must be one of {self.age_labels}"
                
                # Predict aging
                aged_face = self.predict_aging(aligned_face, age_idx)
                if aged_face is None:
                    return "Error generating age progression."
                
                # Convert to PIL Image
                aged_pil = Image.fromarray(aged_face)
                
                # Resize original for comparison
                orig_resized = input_image.resize((self.crop_size, self.crop_size))
                
                # Create comparison image
                comparison = Image.new('RGB', (self.crop_size * 2, self.crop_size))
                comparison.paste(orig_resized, (0, 0))
                comparison.paste(aged_pil, (self.crop_size, 0))
                
                return comparison
        except Exception as e:
            print(f"Error in processing pipeline: {e}")
            return f"An error occurred: {str(e)}"
        finally:
            if os.path.exists(tmp.name):
                os.remove(tmp.name)

def launch_advanced_app():
    """Launch the advanced face aging application"""
    # Initialize with a pre-trained model if available
    app = AdvancedFaceAger("advanced_face_aging_model.h5")
    
    # Create Gradio interface
    iface = gr.Interface(
        fn=app.process_image_pipeline,
        inputs=[
            gr.Image(label="Upload Face Photo", type="pil"),
            gr.Dropdown(
                choices=app.age_labels,
                label="Target Age Group",
                value="50-60"
            )
        ],
        outputs=gr.Image(label="Age Progression Comparison"),
        title="Advanced AI Face Aging",
        description=(
            "Upload a clear front-facing photo to see realistic age progression. "
            "For best results, use a well-lit photo with neutral expression."
        ),
        examples=[
            ["example_young.jpg", "40-50"],
            ["example_adult.jpg", "70-80"]
        ],
        allow_flagging="never"
    )
    
    # Launch with better configuration
    iface.launch(
        server_name="0.0.0.0",
        server_port=7860,
        enable_queue=True,
        share=False
    )

if __name__ == "__main__":
    # Check for GPU availability
    if tf.test.is_gpu_available():
        print("GPU available - using GPU acceleration")
    else:
        print("No GPU detected - running in CPU mode (may be slow)")
    
    # Launch the application
    launch_advanced_app()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
No GPU detected - running in CPU mode (may be slow)


RuntimeError: Unable to open shape_predictor_68_face_landmarks.dat

In [9]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Concatenate, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from PIL import Image
import gradio as gr
import dlib
import face_alignment
from skimage.transform import estimate_transform, warp
from typing import Tuple, Optional, List
import tempfile
import os

class AdvancedFaceAger:
    def __init__(self, model_path: str = None):
        """Initialize the advanced face aging system"""
        # Initialize face detection and alignment
        self.detector = dlib.get_frontal_face_detector()
        
        # Load shape predictor (updated path handling)
        predictor_path = "shape_predictor_68_face_landmarks.dat"
        if not os.path.exists(predictor_path):
            raise FileNotFoundError(
                "dlib shape predictor file not found. "
                "Please download from: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
            )
        self.predictor = dlib.shape_predictor(predictor_path)
        
        # Initialize face alignment with GPU check
        gpu_available = len(tf.config.list_physical_devices('GPU')) > 0
        self.fa = face_alignment.FaceAlignment(
            face_alignment.LandmarksType.TWO_D, 
            flip_input=False,
            device='cuda' if gpu_available else 'cpu'
        )
        
        # Age configuration
        self.age_bins = [
            (20, 30), (30, 40), (40, 50), 
            (50, 60), (60, 70), (70, 80)
        ]
        self.age_labels = [f"{start}-{end}" for start, end in self.age_bins]
        
        # Load or create model
        self.generator = self.load_or_build_model(model_path)
        
        # Image processing parameters
        self.img_size = 256
        self.crop_size = 224
        self.scale = 1.6

    # [Rest of your class methods remain the same...]

def launch_advanced_app():
    """Launch the advanced face aging application"""
    try:
        # Check for GPU availability
        gpu_available = len(tf.config.list_physical_devices('GPU')) > 0
        if gpu_available:
            print("GPU available - using GPU acceleration")
            # Configure GPU memory growth
            gpus = tf.config.experimental.list_physical_devices('GPU')
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        else:
            print("No GPU detected - running in CPU mode (may be slow)")
        
        # Initialize with a pre-trained model if available
        app = AdvancedFaceAger("advanced_face_aging_model.h5")
        
        # Create Gradio interface
        iface = gr.Interface(
            fn=app.process_image_pipeline,
            inputs=[
                gr.Image(label="Upload Face Photo", type="pil"),
                gr.Dropdown(
                    choices=app.age_labels,
                    label="Target Age Group",
                    value="50-60"
                )
            ],
            outputs=gr.Image(label="Age Progression Comparison"),
            title="Advanced AI Face Aging",
            description=(
                "Upload a clear front-facing photo to see realistic age progression. "
                "For best results, use a well-lit photo with neutral expression."
            ),
            allow_flagging="never"
        )
        
        # Launch with better configuration
        iface.launch(
            server_name="0.0.0.0",
            server_port=7860,
            enable_queue=True,
            share=False
        )
    except Exception as e:
        print(f"Failed to launch application: {str(e)}")
        print("Possible solutions:")
        print("1. Make sure you've downloaded shape_predictor_68_face_landmarks.dat")
        print("2. Check that all required packages are installed")
        print("3. Verify you have proper permissions to access the model files")

if __name__ == "__main__":
    launch_advanced_app()

No GPU detected - running in CPU mode (may be slow)
Failed to launch application: dlib shape predictor file not found. Please download from: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Possible solutions:
1. Make sure you've downloaded shape_predictor_68_face_landmarks.dat
2. Check that all required packages are installed
3. Verify you have proper permissions to access the model files
